In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gurobipy as gp
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
%matplotlib inline

In [2]:
train = pd.read_csv("training_data.csv")
train.insert(1, 'X0', [1]*len(train))
test = pd.read_csv("test_data.csv")
test.insert(1, 'X0', [1]*len(test))
train.shape, test.shape

((250, 52), (50, 52))

In [3]:
train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50
0,8.536145,1,-1.535413,0.718888,-2.099149,-0.442842,-0.598978,-1.642574,0.207755,0.760642,...,0.361866,1.793098,-0.631287,-0.061751,0.511049,0.488754,-0.612772,-0.471045,-1.139781,-0.260773
1,4.808344,1,-1.734609,0.551981,-2.147673,-1.552944,1.514910,-1.143972,0.737594,1.321243,...,-0.677985,-0.165679,0.065405,0.137162,1.258197,-0.120834,-1.564834,-0.242565,-0.001827,1.187453
2,-1.530427,1,0.097257,0.107634,-0.194222,0.335454,-0.408199,0.133265,0.706179,0.394971,...,1.108801,0.333791,0.282055,-1.086294,-0.115354,0.257857,-0.088838,-0.751231,1.450609,0.290593
3,-0.428243,1,-0.067702,0.557836,0.700848,-1.121376,1.722274,0.613525,0.700909,-0.417976,...,0.692511,-0.350990,0.624558,0.434520,-0.367409,-1.144681,-0.136524,-0.557214,0.416303,0.484495
4,0.566694,1,0.488729,0.211483,0.568389,0.646837,0.163868,-0.002152,0.125137,0.493571,...,-0.000605,1.075280,0.182281,-1.138458,0.106092,0.544640,-0.383487,-0.425773,2.667647,-0.050748


# Shuffling training set

In [4]:
train = train.sample(frac=1,random_state=42).reset_index(drop=True)

In [5]:
train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50
0,2.626382,1,0.797831,-0.429984,-0.014868,2.941689,-2.670434,0.909308,0.477377,-1.776666,...,-1.254220,-0.539482,-0.753463,-2.494354,-0.875289,0.507418,0.726037,0.421756,0.799545,-0.734498
1,-0.515562,1,0.945354,-0.703539,0.529603,0.000651,-0.551506,-0.556061,0.001207,-0.316895,...,-0.310170,-1.447392,-0.584738,0.081360,-0.186652,-0.402984,-0.137344,-0.192550,0.467632,0.342366
2,6.322560,1,-0.659515,1.115909,0.980689,-1.344514,0.063781,0.462690,0.866678,-0.341734,...,0.690706,1.156570,0.984029,0.545492,0.545874,-0.685298,1.476782,-0.150188,2.307750,1.186673
3,0.283937,1,0.632365,-0.340640,-0.873815,0.097098,-0.988647,0.341962,-0.512534,-0.258885,...,1.407545,-0.205331,-0.168102,0.091085,0.810829,1.120151,-0.816021,0.587934,-1.079718,-0.703395
4,3.807360,1,0.200424,0.189315,-0.905100,-2.854128,1.319839,1.724242,-1.311053,-0.556105,...,-1.332689,-0.585949,0.927983,1.589170,2.029222,1.220493,-2.512068,0.392134,-0.779833,-1.001974


# Gurobi Solve function

In [6]:
def solveQuadratic(m_var, A, sense, b, lb = None, ub = None, min_max = 'maximize', vtype = None, Q = None, L = None, C = 0, show_output = False, time_limit = 3600):

    
    # Model initialization
    model = gp.Model()
    
    # Variables initialization
    modelX = model.addMVar(m_var, lb=None, ub=None, vtype=vtype)
    
    # Constraint intialization
    modelConstr = model.addMConstr(A, modelX, sense, b)
    
    #  objective init
    if min_max == 'maximize':
        model.setMObjective(Q,L,C,sense=gp.GRB.MAXIMIZE)
    if min_max == 'minimize':
        model.setMObjective(Q,L,C, sense = gp.GRB.MINIMIZE)
    
    model.Params.OutputFlag = 0 # restricting output jargon
    model.Params.timeLimit = time_limit
    
    model.optimize()
    
    if show_output:
        print("\nobjective value")
        print(model.objVal)
        print('-----------------------------------')

        print("\nvalues of variable X")
        print(modelX.x)

        print('-----------------------------------')
    
    return modelX.x

# Direct Variable selection - MIQP

In [7]:
import random

# Split a dataset into k folds
def cross_validation_split(data_ix, folds=10, seed=42):
#     print(data_ix)
    dataset_split = list()
    dataset_copy = data_ix.copy()
    fold_size = int(len(data_ix) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size and len(dataset_copy) > 0:
            index = dataset_copy[0]
            fold.append(dataset_copy.pop(0))
#         print(fold)
        dataset_split.append(fold)
    return dataset_split

# train_ix = list(train.index.values)
# split_ix = cross_validation_split(train_ix, seed=42)

In [8]:
def LassoRegressionMIQPSolve(X, y, k, M, m, time_limit):
    
    ## Objective Definition
    # Quadratic part
    q = 2*m+1
    Q = np.zeros((q,q))
    Q[0:m+1, 0:m+1] = X.T @ X
    
    
    # Linear Part
    L = np.array(list(-2 * y.T @ X) + [0]*m)
#     pd.DataFrame(L).to_csv("L.csv")
    
    ## Constraint Definition
    A = np.zeros((q,q))
    sense = np.array(['']*q)
    b = np.array([0]*q)
    
    row = 0
    # -Mz_j <= b_j
    A[row:row+m, 1:m+1] = np.identity(m)
    A[row:row+m, m+1:q] = M*np.identity(m)
    sense[row:row+m] = ['>']*m
    b[row:row+m] = [0]*m    
    
    row+=m
    # b_j <= Mz_j 
    A[row:row+m, 1:m+1] = np.identity(m)
    A[row:row+m, m+1:q] = -1*M*np.identity(m)
    sense[row:row+m] = ['<']*m
    b[row:row+m] = [0]*m
    
    row+=m
    # sum(z_j) = k
    A[row, m+1:q] = [1]*m
    sense[row] = '<'
    b[row] = k
    
    
    beta_vals = solveQuadratic(m_var=q, A=A, sense=sense, b=b, lb=np.array([-M]*q), ub=np.array([M]*q), min_max='minimize', vtype = ['C']*(m+1) + ['B']*m, time_limit=time_limit, Q=Q, L=L, C=0)
    
    return beta_vals

In [9]:
def LassoRegressionMIQP(train, split_ix, train_ix, k_range):
    
    mse_outer = {}
    m = train.shape[1] - 1
    
    for k in k_range:
        mse_inner = []
        for i, cv_test_ix in enumerate(split_ix):
            cv_train_ix = list(set(train_ix) - set(cv_test_ix))
            cv_train, cv_test = train.iloc[cv_train_ix], train.iloc[cv_test_ix]

            y_train, X_train = cv_train['y'].values, cv_train.drop('y', axis = 1).values
            y_val, X_val = cv_test['y'].values, cv_test.drop('y', axis = 1).values

            beta_vals = LassoRegressionMIQPSolve(X = X_train, y = y_train, k = k, M = 2, m=X_train.shape[1]-1, time_limit = 3600)
            
            pd.DataFrame(beta_vals).to_csv(f'beta_vals_{k}.csv')
            
            se = (X_val @ beta_vals[0:m] - y_val).T @ (X_val @ beta_vals[0:m] - y_val)
            mse = se / len(y_val)
            mse_inner.append(mse)
            
        mse_outer[k] = np.mean(mse_inner)
    
    return mse_outer
    

In [10]:
train_ix = list(train.index.values)
split_ix = cross_validation_split(train_ix, seed=42)

y_train, X_train = train['y'].values, train.drop('y', axis = 1).values
y_test, X_test = test['y'].values, test.drop('y', axis = 1).values

m=train.shape[1]-2
mse_l = LassoRegressionMIQP(train, split_ix, train_ix, range(1, m+1))

Academic license - for non-commercial use only - expires 2022-11-04
Using license file C:\Users\mehen\gurobi.lic


# Best value of K

In [11]:
best_k = pd.Series(mse_l).idxmin()
best_k

23

# Solving Lasso Regression using best K

In [12]:
m = train.shape[1]-2

beta_vals = LassoRegressionMIQPSolve(X = X_train, y = y_train, k = best_k, M = 2, m=m, time_limit = 3600)

se_test = (X_test @ beta_vals[0:m+1] - y_test).T @ (X_test @ beta_vals[0:m+1] - y_test)
mse_test = se_test / len(y_test)

In [13]:
mse_test

6.579258983715103

# Q4 : Lasso Regression using Scikit-learn

In [14]:
cv = KFold(n_splits=10, shuffle=False)
cv.get_n_splits(train)

10

In [52]:
lasso = LassoCV(cv=cv, random_state=42, fit_intercept=False).fit(X_train, y_train)

In [53]:
reg = lasso.alpha_
reg

0.07753299509155244

In [54]:
se_test = (X_test @ lasso.coef_ - y_test).T @ (X_test @ lasso.coef_ - y_test)

In [55]:
mse_test = se_test / len(y_test)
mse_test

2.369982855564395

In [58]:
n_features = sum(lasso.coef_ != 0) - 1
n_features

19

In [59]:
len(lasso.coef_)

51

In [37]:
X_train[1,1]

0.945354290688534

In [62]:
sum(lasso.coef_ == 0)

31